In [3]:
#treel.py
import requests
try:
    import xml.etree.cElementTree as ET
except Importerror:
    import xml.etree.ElementTree as ET
    
content = requests.get('http://invoice.etax.nat.gov.tw/invoice.xml')
tree = ET.fromstring(content.text)

item = tree[0].find('item')
print("find 方法: "+item[0].text)

item = tree[0].findall('item')
print("findall 方法: "+item[0][0].text)

item = list(tree.iter(tag="item"))
print("find 方法: "+item[0][0].text)

find 方法: 110年03月、04
findall 方法: 110年03月、04
find 方法: 110年03月、04


In [3]:
import requests
try:
    import xml.etree.cElementTree as ET
except Importerror:
    import xml.etree.ElementTree as ET

In [5]:
content = requests.get('http://invoice.etax.nat.gov.tw/invoice.xml')
tree = ET.fromstring(content.text)  #解析XML
items = list(tree.iter(tag='item'))  #取得item標籤內容
title = items[0][0].text  #期別
ptext = items[0][2].text  #中獎號碼
#將中獎號碼字串中的<p>及</p>替換為「\n」換行符號
ptext = ptext.replace('<p>','').replace('</p>','\n')
print(title + '月\n' + ptext[:-1])  #ptext[:-1]為移除最後一個\n


110年03月、04月
特別獎：59518250
特獎：81016847
頭獎：22884739、80660537、62637675
增開六獎：187


In [11]:
from flask import Flask
app = Flask(__name__)

from flask import Flask, request, abort
from linebot import  LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextSendMessage, TextMessage

import requests
try:
    import xml.etree.cElementTree as ET
except ImportError:
    import xml.etree.ElementTree as ET

line_bot_api = LineBotApi('5VNonF2HXNCldN5goyRV97kmThATOsMCqVqxBeF1HGbg+VVlMUG5jBtDdp6MkNMWCJ0CcvVobqIaCdJBlB12D+bXBFAon0xd9tR5QyknypAwWatIRblIjbQ2vzzArJI9utUCvlhPJ2hYBKwtx+JNtAdB04t89/1O/w1cDnyilFU=')
handler = WebhookHandler('777ba4a2a8724ceb18e091a72ed7ebb2')


@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    mtext = event.message.text
    if mtext == '@本期中獎號碼':
        try:
            message = TextSendMessage(
            text = monoNum(0))
            line_bot_api.reply_message(event.reply_token,message)
            monoNum(n)
        except:
            line_bot_api.reply_message(event.reply_token,TextSendMessage(text='讀取發票號碼發生錯誤！'))

    elif mtext == '@前期中獎號碼':
        try:
            #加入前期與前前期的期別，呼叫mononum函式，取得前期中獎號碼。
            #兩期中獎號碼中間可以加入一個空白列「\n\n」，較易閱讀。
            message = TextSendMessage( text = monoNum(1)+'\n\n'+ monoNum(2))        
            line_bot_api.reply_message(event.reply_token,message)
        except:
            line_bot_api.reply_message(event.reply_token,TextSendMessage(text='讀取發票號碼發生錯誤！'))

    elif mtext == '@輸入發票最後三碼':
        line_bot_api.reply_message(event.reply_token,TextSendMessage(text='請輸入發票最後三碼進行對獎！'))
    
    elif (mtext.isdigit() and len(mtext) == 3): #判斷輸入的文字長度是3 以及 是數字
        try:
            content = requests.get('http://invoice.etax.nat.gov.tw/invoice.xml')
            tree = ET.fromstring(content.text)  #解析XML
            items = list(tree.iter(tag='item'))  #取得item標籤內容
            #取處中獎號碼，並且存到變數ptext
            ptext = items[0][2].text
            #line_bot_api.reply_message(event.reply_token,TextSendMessage(text='安安！'))
            ptext = ptext.replace('<p>','').replace('</p>','' ) #將<p>用空白取代
            temlist = ptext.split('：') #將ptext用广:」切割,並存人templist
            prizelist = []  #別獎或特獎後三码
            prizelist.append(temlist[1][5:8])
            prizelist.append(temlist[2][5:8])
            for i in range(3): #頭獎後三碼
                prizelist.append(temlist[3][9*i+5:9*i+8])
            sixlist = temlist[4].split('、')#增開六獎
            for i in range(len(sixlist)):
                prizelist.append(sixlist[i])
            if mtext in prizelist:
                message = '符合某獎項後三碼,請自行核對發票前五碼!\n\n'
                message += monoNum(0)
            else:
                message ='很可惜,未中獎。請輸入下一張發票最後三碼。'
            line_bot_api.reply_message(event.reply_token,TextSendMessage(text=message))
        except:
            #LINEBOT回覆「讀取發票號碼發生錯誤！」
            line_bot_api.reply_message(event.reply_token,TextSendMessage(text='讀取發票號碼發生錯誤！'))
    else:
        line_bot_api.reply_message(event.reply_token,TextSendMessage(text='請輸入發票最後三碼進行對獎！'))

def monoNum(n):
    content = requests.get('http://invoice.etax.nat.gov.tw/invoice.xml')
    tree = ET.fromstring(content.text)  #解析XML
    items = list(tree.iter(tag='item'))  #取得item標籤內容
    title = items[n][0].text  #期別
    ptext = items[n][2].text  #中獎號碼
    #將中獎號碼字串中的<p>及</p>替換為「\n」換行符號
    ptext = ptext.replace('<p>','').replace('</p>','\n')
    return title + '月\n' + ptext[:-1]  #ptext[:-1]為移除最後一個\n


if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
[2021-07-16 13:43:58,978] ERROR in app: Exception on /callback [POST]
Traceback (most recent call last):
  File "<ipython-input-11-6165e5ca479c>", line 37, in handle_message
    monoNum(n)
NameError: name 'n' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "D:\Anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "D:\Anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "D:\Anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "D:\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "D:\Anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "D:\Anacon

In [3]:
from flask import Flask
app = Flask(__name__)

from flask import Flask, request, abort
from linebot import  LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextSendMessage, TextMessage, FlexSendMessage
from linebot.models.flex_message import BubbleContainer

import requests
try:
    import xml.etree.cElementTree as ET
except ImportError:
    import xml.etree.ElementTree as ET

line_bot_api = LineBotApi('5VNonF2HXNCldN5goyRV97kmThATOsMCqVqxBeF1HGbg+VVlMUG5jBtDdp6MkNMWCJ0CcvVobqIaCdJBlB12D+bXBFAon0xd9tR5QyknypAwWatIRblIjbQ2vzzArJI9utUCvlhPJ2hYBKwtx+JNtAdB04t89/1O/w1cDnyilFU=')
handler = WebhookHandler('777ba4a2a8724ceb18e091a72ed7ebb2')


@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    mtext = event.message.text
    if mtext == '@本期中獎號碼':
        try:
            message = FlexSendMessage(
                alt_text = monoNum(0),
                contents = invo(monoNum(0))
            )
            line_bot_api.reply_message(event.reply_token,message)
        except:
            line_bot_api.reply_message(event.reply_token,FlexSendMessage(text='讀取發票號碼發生錯誤！'))

    elif mtext == '@前期中獎號碼':
        try:
            #加入前期與前前期的期別，呼叫mononum函式，取得前期中獎號碼。
            #兩期中獎號碼中間可以加入一個空白列「\n\n」，較易閱讀。
            message = FlexSendMessage(
                #text = monoNum(1)+'\n\n'+ monoNum(2)
                alt_text = monoNum(1)+'\n\n'+ monoNum(2),
                contents = invo1(monoNum(1),monoNum(2))
            )        
            line_bot_api.reply_message(event.reply_token,message)   
        except:
            line_bot_api.reply_message(event.reply_token,FlexSendMessage(text='讀取發票號碼發生錯誤！'))

    elif mtext == '@輸入發票最後三碼':
        line_bot_api.reply_message(event.reply_token,TextSendMessage(text='請輸入發票最後三碼進行對獎！'))
    
    elif (mtext.isdigit() and len(mtext) == 3): #判斷輸入的文字長度是3 以及 是數字
        try:
            content = requests.get('http://invoice.etax.nat.gov.tw/invoice.xml')
            tree = ET.fromstring(content.text)  #解析XML
            items = list(tree.iter(tag='item'))  #取得item標籤內容
            #取處中獎號碼，並且存到變數ptext
            ptext = items[0][2].text
            #line_bot_api.reply_message(event.reply_token,TextSendMessage(text='安安！'))
            ptext = ptext.replace('<p>','').replace('</p>','' ) #將<p>用空白取代
            temlist = ptext.split('：') #將ptext用广:」切割,並存人templist
            prizelist = []  #別獎或特獎後三码
            prizelist.append(temlist[1][5:8])
            prizelist.append(temlist[2][5:8])
            for i in range(3): #頭獎後三碼
                prizelist.append(temlist[3][9*i+5:9*i+8])
            sixlist = temlist[4].split('、')#增開六獎
            for i in range(len(sixlist)):
                prizelist.append(sixlist[i])
            if mtext in prizelist:
                message = '符合某獎項後三碼,請自行核對發票前五碼!\n\n'
                message += monoNum(0)
            else:
                message ='很可惜,未中獎。請輸入下一張發票最後三碼。'
            line_bot_api.reply_message(event.reply_token,TextSendMessage(text=message))
        except:
            #LINEBOT回覆「讀取發票號碼發生錯誤！」
            line_bot_api.reply_message(event.reply_token,TextSendMessage(text='讀取發票號碼發生錯誤！'))
    else:
        line_bot_api.reply_message(event.reply_token,TextSendMessage(text='請輸入發票最後三碼進行對獎！'))

def monoNum(n):
    content = requests.get('http://invoice.etax.nat.gov.tw/invoice.xml')
    tree = ET.fromstring(content.text)  #解析XML
    items = list(tree.iter(tag='item'))  #取得item標籤內容
    title = items[n][0].text  #期別
    ptext = items[n][2].text  #中獎號碼
    #將中獎號碼字串中的<p>及</p>替換為「\n」換行符號
    ptext = ptext.replace('<p>','').replace('</p>','\n')
    return title + '月\n' + ptext[:-1]  #ptext[:-1]為移除最後一個\n

def invo(x):
    contents = {
      "type": "bubble",
      "body": {
        "type": "box",
        "layout": "vertical",
        "contents": [
          {
            "type": "text",
            "text": "Invoice",
            "weight": "bold",
            "color": "#1DB446",
            "size": "3xl"
          },
          {
            "type": "text",
            "text": x[0:11],
            "weight": "bold",
            "size": "xxl",
            "margin": "md"
          },
          {
            "type": "text",
            "text": x[11:24],
            "size": "xs",
            "color": "#EA0000",
            "wrap": True
          },
          {
            "type": "box",
            "layout": "vertical",
            "margin": "xxl",
            "spacing": "sm",
            "contents": [
              {
                "type": "separator",
                "margin": "sm"
              },
              {
                "type": "box",
                "layout": "horizontal",
                "margin": "xxl",
                "contents": [
                  {
                    "type": "text",
                    "text": "特獎",
                    "size": "sm",
                    "color": "#555555"
                  },
                  {
                    "type": "text",
                    "text": x[28:36],
                    "size": "sm",
                    "color": "#111111",
                    "align": "end"
                  }
                ]
              },
              {
                "type": "box",
                "layout": "horizontal",
                "contents": [
                  {
                    "type": "text",
                    "text": "頭獎",
                    "size": "sm",
                    "color": "#555555"
                  },
                  {
                    "type": "text",
                    "text": x[40:48],
                    "size": "sm",
                    "color": "#111111",
                    "align": "end"
                  }
                ]
              },
              {
                "type": "box",
                "layout": "horizontal",
                "contents": [
                  {
                    "type": "text",
                    "text": "頭獎",
                    "size": "sm",
                    "color": "#555555"
                  },
                  {
                    "type": "text",
                    "text": x[49:57],
                    "size": "sm",
                    "color": "#111111",
                    "align": "end"
                  }
                ]
              },
              {
                "type": "box",
                "layout": "horizontal",
                "contents": [
                  {
                    "type": "text",
                    "text": "頭獎",
                    "size": "sm",
                    "color": "#555555"
                  },
                  {
                    "type": "text",
                    "text": x[58:66],
                    "size": "sm",
                    "color": "#111111",
                    "align": "end"
                  }
                ]
              }
            ]
          },
          {
            "type": "separator",
            "margin": "xxl"
          },
          {
            "type": "box",
            "layout": "horizontal",
            "margin": "md",
            "contents": [
              {
                "type": "text",
                "text": "增開六獎",
                "size": "xs",
                "color": "#2894FF",
                "flex": 0
              },
              {
                "type": "text",
                "text": x[72:75],
                "color": "#2894FF",
                "size": "xs",
                "align": "end"
              }
            ]
          }
        ]
      },
      "styles": {
        "footer": {
          "separator": True
        }
      }
    }
    return contents

def invo1(x,y):
    contents = {
      "type": "carousel",
      "contents": [
        {
          "type": "bubble",
          "body": {
            "type": "box",
            "layout": "vertical",
            "contents": [
              {
                "type": "text",
                "text": "Invoice",
                "size": "3xl",
                "color": "#1DB446",
                "weight": "bold"
              },
              {
                "type": "text",
                "text": x[0:11],
                "size": "xxl",
                "weight": "bold",
                "margin": "md"
              },
              {
                "type": "text",
                "text": x[11:24],
                "size": "xs",
                "color": "#EA0000"
              },
              {
                "type": "box",
                "layout": "vertical",
                "contents": [
                  {
                    "type": "separator",
                    "margin": "sm"
                  },
                  {
                    "type": "box",
                    "layout": "horizontal",
                    "contents": [
                      {
                        "type": "text",
                        "text": "特獎",
                        "size": "sm",
                        "color": "#555555"
                      },
                      {
                        "type": "text",
                        "text": x[28:36],
                        "size": "sm",
                        "color": "#111111",
                        "align": "end"
                      }
                    ],
                    "margin": "xxl"
                  },
                  {
                    "type": "box",
                    "layout": "horizontal",
                    "contents": [
                      {
                        "type": "text",
                        "text": "頭獎",
                        "size": "sm",
                        "color": "#555555"
                      },
                      {
                        "type": "text",
                        "text": x[40:48],
                        "size": "sm",
                        "color": "#111111",
                        "align": "end"
                      }
                    ],
                    "margin": "xxl"
                  },
                  {
                    "type": "box",
                    "layout": "horizontal",
                    "contents": [
                      {
                        "type": "text",
                        "text": "頭獎",
                        "size": "sm",
                        "color": "#555555"
                      },
                      {
                        "type": "text",
                        "text": x[49:57],
                        "size": "sm",
                        "color": "#111111",
                        "align": "end"
                      }
                    ],
                    "margin": "xxl"
                  },
                  {
                    "type": "box",
                    "layout": "horizontal",
                    "contents": [
                      {
                        "type": "text",
                        "text": "頭獎",
                        "size": "sm",
                        "color": "#555555"
                      },
                      {
                        "type": "text",
                        "text": x[58:66],
                        "size": "sm",
                        "color": "#111111",
                        "align": "end"
                      }
                    ],
                    "margin": "xxl"
                  }
                ],
                "margin": "xxl",
                "spacing": "sm"
              },
              {
                "type": "separator",
                "margin": "xxl"
              },
              {
                "type": "box",
                "layout": "horizontal",
                "contents": [
                  {
                    "type": "text",
                    "text": "增開六獎",
                    "flex": 0,
                    "size": "xs",
                    "color": "#0080FF"
                  },
                  {
                    "type": "text",
                    "text": x[72:75],
                    "color": "#0080FF",
                    "size": "xs",
                    "align": "end"
                  }
                ],
                "margin": "md"
              }
            ]
          }
        },
        {
          "type": "bubble",
          "body": {
            "type": "box",
            "layout": "vertical",
            "contents": [
              {
                "type": "text",
                "text": "Invoice",
                "size": "3xl",
                "color": "#1DB446",
                "weight": "bold"
              },
              {
                "type": "text",
                "text": y[0:11],
                "size": "xxl",
                "weight": "bold",
                "margin": "md"
              },
              {
                "type": "text",
                "text": y[11:24],
                "size": "xs",
                "color": "#EA0000"
              },
              {
                "type": "box",
                "layout": "vertical",
                "contents": [
                  {
                    "type": "separator",
                    "margin": "sm"
                  },
                  {
                    "type": "box",
                    "layout": "horizontal",
                    "contents": [
                      {
                        "type": "text",
                        "text": "特獎",
                        "size": "sm",
                        "color": "#555555"
                      },
                      {
                        "type": "text",
                        "text": y[28:36],
                        "size": "sm",
                        "color": "#111111",
                        "align": "end"
                      }
                    ],
                    "margin": "xxl"
                  },
                  {
                    "type": "box",
                    "layout": "horizontal",
                    "contents": [
                      {
                        "type": "text",
                        "text": "頭獎",
                        "size": "sm",
                        "color": "#555555"
                      },
                      {
                        "type": "text",
                        "text": y[40:48],
                        "size": "sm",
                        "color": "#111111",
                        "align": "end"
                      }
                    ],
                    "margin": "xxl"
                  },
                  {
                    "type": "box",
                    "layout": "horizontal",
                    "contents": [
                      {
                        "type": "text",
                        "text": "頭獎",
                        "size": "sm",
                        "color": "#555555"
                      },
                      {
                        "type": "text",
                        "text": y[49:57],
                        "size": "sm",
                        "color": "#111111",
                        "align": "end"
                      }
                    ],
                    "margin": "xxl"
                  },
                  {
                    "type": "box",
                    "layout": "horizontal",
                    "contents": [
                      {
                        "type": "text",
                        "text": "頭獎",
                        "size": "sm",
                        "color": "#555555"
                      },
                      {
                        "type": "text",
                        "text": y[58:66],
                        "size": "sm",
                        "color": "#111111",
                        "align": "end"
                      }
                    ],
                    "margin": "xxl"
                  }
                ],
                "margin": "xxl",
                "spacing": "sm"
              },
              {
                "type": "separator",
                "margin": "xxl"
              },
              {
                "type": "box",
                "layout": "horizontal",
                "contents": [
                  {
                    "type": "text",
                    "text": "增開六獎",
                    "flex": 0,
                    "size": "xs",
                    "color": "#0080FF"
                  },
                  {
                    "type": "text",
                    "text": y[72:75],
                    "color": "#0080FF",
                    "size": "xs",
                    "align": "end"
                  }
                ],
                "margin": "md"
              }
            ]
          }
        }
      ]
    }
    return contents

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [28/Jul/2021 10:22:27] "POST /callback HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2021 10:22:33] "POST /callback HTTP/1.1" 200 -
